### 텍스트 분류 모델 학습 시키기
- 한국어 기사 제목을 바탕으로 기사의 카테고리를 분류하는 텍스트 분류 모델 학습 실습
- 출처: LLM을 활용한 실전 AI 애플리케이션 개발

In [82]:
# 데이터 준비
# 데이터 출처: 허깅페이스 KLUE 데이터셋의 YNAT 서브셋: 연합 뉴스 기사의 제목과 기사가 속한 카테고리 정보가 담겨져 있음.
# !pip install datasets
from datasets import load_dataset
klue_tc_train = load_dataset('klue', 'ynat', split='train')
klue_tc_eval = load_dataset('klue', 'ynat', split='validation')

In [83]:
klue_tc_train
klue_tc_train[0]

{'guid': 'ynat-v1_train_00000',
 'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영',
 'label': 3,
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947',
 'date': '2016.06.30. 오전 10:36'}

In [84]:
klue_tc_train.features
klue_tc_train.features['label'].names

['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']

In [85]:
# 불필요한 컬럼 제거
klue_tc_train = klue_tc_train.remove_columns(['guid', 'url', 'date'])
klue_tc_eval = klue_tc_eval.remove_columns(['guid', 'url', 'date'])

In [86]:
klue_tc_train

Dataset({
    features: ['title', 'label'],
    num_rows: 45678
})

In [167]:
klue_tc_train.features['label'].names  # 카테고리 이름과 해당 카테고리 ID 라벨이 연결되어있는 ClassLabel 객체가 존재

['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']

In [88]:
klue_tc_train.features['label'].int2str(1)  # int2str 메서드에 숫자를 입력하면 해당 ID의 카테고리를 반환

'경제'

In [89]:
# 라벨 ID -> 카테고리 이름으로 변환해서 'label_str'라는 새 컬럼에 추가
klue_tc_label = klue_tc_train.features['label']

def make_str_label(batch):
    batch['label_str'] = klue_tc_label.int2str(batch['label'])
    return batch

klue_tc_train = klue_tc_train.map(make_str_label, batched=True, batch_size=1000)

In [90]:
klue_tc_train[100]

{'title': '트레이드 성사한 잠실구장 두 사령탑 불펜 투수가 필요했다', 'label': 5, 'label_str': '스포츠'}

In [102]:
# 실습을 위해서 train: 10000 / eval: 1000 / test: 1000개만 사용
train_dataset = klue_tc_train.train_test_split(test_size=10000, shuffle=True, seed=42)['test'] # 보통 test_size에 분리할 비율을 적는데 정수(10000)를 입력하면 train = (train - 10000)개 / test = 10000개로 분리한 것 중 test만 쓰겠다.(뒤에 ['test'])
valid_dataset = klue_tc_train.train_test_split(test_size=1000, shuffle=True, seed=42)['test']  # train 데이터셋 중에 1000개를 랜덤 추출 후 'test'로 분할해서 학습 중 eval 데이터로 사용

# 모델 학습 후 성능 평가용 테스트 데이터
test_dataset = klue_tc_eval.train_test_split(test_size=1000, shuffle=True, seed=42)['test']  # validation 데이터셋 중 1000개를 'test'로 분할해서 평가용 test 데이터로 사용

In [103]:
train_dataset
valid_dataset
test_dataset

Dataset({
    features: ['title', 'label'],
    num_rows: 1000
})

### 트레이너 API를 활용한 학습

In [48]:
# 트레이너 API를 활용한 학습
import torch
import numpy as np
from transformers import (
    Trainer,
    TrainingArguments,
    AutoModelForSequenceClassification,
    AutoTokenizer
)

def tokenize_func(examples):
    return tokenizer(examples["title"], padding="max_length", truncation=True)

model_id = "klue/roberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=len(train_dataset.features['label'].names))
# "klue/roberta-base" 같이 바디만 있는 모델을 헤드(텍스트 분류, 텍스트 재귀적 생성 등)를 포함해서 불러오는 AutoModelForSequenceClassification 라이브러리로 모델을 불러올 경우
# 헤드 부분이 랜덤으로 초기화 되어서 분류 헤드의 분류 클래스 수를 알려주기 위해
# num_labels=len(train_dataset.features['label'].names) 를 입력해 클래스 수를 전달 모델에 전달 함.

tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [50]:
# 데이터셋 토큰화 작업  # 허깅페이스의 map 메서드를 활용해 배치 작업으로 수행
train_dataset = train_dataset.map(tokenize_func, batched=True)
valid_dataset = valid_dataset.map(tokenize_func, batched=True)
test_dataset = test_dataset.map(tokenize_func, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [54]:
test_dataset
# token_type_ids는 BERT 모델에서 문장의 구분 ID (일반적으로 단일 문장이라면 0, 두 개의 문장이면 [0, 0, 0, ..., 1, 1, 1])
# attention_mask는 패딩 여부를 나타내는 마스크 (1=실제 토큰, 0=패딩)

Dataset({
    features: ['title', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

In [51]:
# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    push_to_hub=False,

)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [55]:
# 평가 함수 정의
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # 예측 결과 중 가장 큰 값을 갖는 클래스를 np.argmax 함수로 뽑아서 저장
    return {"accuracy": (predictions == labels).mean()}  # predictions와 정답(label)이 일치하는 비율을 정확도(accuracy)로 결과 딕셔너리에 저장해 반환

In [56]:
# 학습 진행
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

trainer.train()

<ipython-input-56-2e3ba26e06e0>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dgriii0606 (dg-test) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.554900,0.311685,0.905000


TrainOutput(global_step=1250, training_loss=0.5898691162109375, metrics={'train_runtime': 342.7309, 'train_samples_per_second': 29.177, 'train_steps_per_second': 3.647, 'total_flos': 2631228672000000.0, 'train_loss': 0.5898691162109375, 'epoch': 1.0})

In [57]:
# test 데이터셋으로 모델 평가
trainer.evaluate(test_dataset)  # 정확도: 0.856

{'eval_loss': 0.4709104597568512,
 'eval_accuracy': 0.856,
 'eval_runtime': 7.4177,
 'eval_samples_per_second': 134.813,
 'eval_steps_per_second': 16.852,
 'epoch': 1.0}

 ### 트레이너 API를 사용하지 않고 학습

In [68]:
import torch
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from transformers import AdamW

# 토크나이징 함수  # 제목 컬럼에 대한 토큰화 진행
def tokenize_func(examples):
    return tokenizer(examples["title"], padding="max_length", truncation=True)

# 모델과 토크나이저 불러오기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 토치 쿠다 실행 함수를 디바이스에 저장
model_id = "klue/roberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=len(train_dataset.features['label'].names))
tokenizer = AutoTokenizer.from_pretrained(model_id)
model.to(device)  # 트레이너 API와는 다르게 직접 모델을 디바이스(쿠다 사용 GPU)에 옮김.

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [62]:
# 데이터 전처리 함수
def make_dataloader(dataset, batch_size, shuffle=True):
    dataset = dataset.map(tokenize_func, batched=True).with_format("torch") # 토큰화 수행
    dataset = dataset.rename_column("label", "labels") # 'label' 컬럼 이름을 'labels'로 변경
    dataset = dataset.remove_columns(column_names=['title']) # 토큰으로 변환 후 필요없는 'title' 텍스트 컬럼 삭제
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle) # 파이토치에서 제공하는 DataLoader 클래스를 사용해서 데이터셋을 배치 데이터로 만듦.

In [63]:
# 데이터로더 만들기
train_dataloader = make_dataloader(train_dataset, batch_size=8, shuffle=True)
valid_dataloader = make_dataloader(valid_dataset, batch_size=8, shuffle=False)
test_dataloader = make_dataloader(test_dataset, batch_size=8, shuffle=False)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [65]:
# 학습을 수행하는 함수
def train_epoch(model, data_loader, optimizer):
    model.train() # 모델 학습
    total_loss = 0 # 평균 손실을 구하기 위한 총 손실 누적용 변수, 0으로 총 손실 초기화
    for batch in tqdm(data_loader):
        optimizer.zero_grad()  # 이전 단계에서 계산된 기울기 초기화(기울기 누적 방지)
        input_ids = batch['input_ids'].to(device)  # 모델에 입력할 토큰 아이디
        attention_mask = batch['attention_mask'].to(device)  # 모델에 입력할 어텐션 마스크
        labels = batch['labels'].to(device)  # 모델에 입력할 레이블
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)  # 모델 계산
        loss = outputs.loss  # 모델의 출력에서 손실
        loss.backward()  # 손실 역전파
        optimizer.step()  # 모델 업데이트
        total_loss += loss.item()  # 총 손실 누적
    avg_loss = total_loss / len(data_loader) # 총 손실을 데이터 갯수로 나누어서 평균 손실 계산
    return avg_loss

In [66]:
# 평가를 위한 함수
# 학습 수행 함수에서 학습 모드 -> 추론 모드로 변경하고, 손실과 함께 출력된 logits 중 가장 큰 값으로 예측한 카테고리 정보 가져와서 실제 정답과 비교해서 정확도 계산
def evaluate(model, data_loader):
    model.eval()  # 모델 추론
    total_loss = 0
    predictions = []  # 예측 클래스를 담을 빈 리스트 생성
    true_labels = []  # 실제 클래스를 담을 빈 리스트 생성
    with torch.no_grad():  # 기울기 계산
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

            logits = outputs.logits  # 모델의 출력에서 로짓
            loss = outputs.loss  # 모델의 출력에서 손실
            total_loss += loss.item() # 손실 누적
            preds = torch.argmax(logits, dim=-1)  # 최종 출력된 로짓 중 큰 값을 갖는 클래스
            predictions.extend(preds.cpu().numpy())  # predictions 리스트에 추가
            true_labels.extend(labels.cpu().numpy())  # true_labels 리스트에 추가
    avg_loss = total_loss / len(data_loader)
    accuracy = np.mean(np.array(predictions) == np.array(true_labels))  # 예측값과 실제값이 같은 비율을 계산
    return avg_loss, accuracy

In [69]:
# for문을 활용해 학습 수행  # train_epoch() 학습하고, evaluate()로 성능 평가
num_epochs = 1
optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습 루프 수행
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss = train_epoch(model, train_dataloader, optimizer)
    print(f"Training loss: {train_loss}")
    valid_loss, valid_accuracy = evaluate(model, valid_dataloader)
    print(f"Validation loss: {valid_loss}")
    print(f"Validation accuracy: {valid_accuracy}")

Epoch 1/1


  0%|          | 0/1250 [00:00<?, ?it/s]

Training loss: 0.6374597485899925


  0%|          | 0/125 [00:00<?, ?it/s]

Validation loss: 0.3188380221128464
Validation accuracy: 0.902


In [70]:
# 모델 테스트
_, test_accuracy = evaluate(model, test_dataloader)
print(f"Test accuracy: {test_accuracy}")  # 정확도: 0.83

  0%|          | 0/125 [00:00<?, ?it/s]

Test accuracy: 0.83



### 학습한 모델 업로드

In [ ]:
from huggingface_hub import login

login(token="****")

In [168]:
# 모델의 예측 아이디와 문자열 레이블을 연결할 데이터를 모델 config에 저장
id2label = {i: label for i, label in enumerate(train_dataset.features['label'].names)}
label2id = {label: i for i, label in id2label.items()}
model.config.id2label = id2label
model.config.label2id = label2id

In [169]:
repo_id = "edgeun/roberta-base-klue-ynat-classification"
trainer.push_to_hub(repo_id)  # Trainer를 사용한 경우

# # Trainer를 사용하지 않고 직접 학습한 경우
# model.push_to_hub(repo_id)
# tokenizer.push_to_hub(repo_id)

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/edgeun/results/commit/2cd3750a4fadbfb7e9462c95d62f38e240f0bf7f', commit_message='edgeun/roberta-base-klue-ynat-classification', commit_description='', oid='2cd3750a4fadbfb7e9462c95d62f38e240f0bf7f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/edgeun/results', endpoint='https://huggingface.co', repo_type='model', repo_id='edgeun/results'), pr_revision=None, pr_num=None)

### 모델 추론하기
- pipeline을 활용한 방법
- 직접 모델과 토크나이저를 불러와 활용하는 방법

In [176]:
# pipeline으로 추론하기
from transformers import pipeline

model_id = "edgeun/roberta-base-klue-title-classification"

model_pipeline = pipeline("text-classification", model=model_id)

config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Device set to use cuda:0


In [188]:
dataset = load_dataset("klue", "ynat", split="validation")

title_text = dataset["title"][:5]  # 예시 뉴스 제목 가져오기
result = model_pipeline(title_text)

# DataFrame 변환
import pandas as pd
df = pd.DataFrame({
    "뉴스 제목": title_text,
    "예측 라벨": [i["label"] for i in result],
    "예측 확률": [i["score"] for i in result]
})

df

,뉴스 제목,예측 라벨,예측 확률
0,5억원 무이자 융자는 되고 7천만원 이사비는 안된다,경제,0.975947
1,왜 수소충전소만 더 멀리 떨어져야 하나 한경연 규제개혁 건의,사회,0.899616
2,항응고제 성분 코로나19에 효과…세포실험서 확인,IT과학,0.958991
3,실거래가 가장 비싼 역세권은 신반포역…3.3㎡당 1억 육박,경제,0.976918
4,기자회견 하는 성 소수자 단체,사회,0.951464


### 직접 추론 클래스 생성

In [185]:
import torch
from torch.nn.functional import softmax
from transformers import AutoModelForSequenceClassification, AutoTokenizer

class CustomPipeline:
    def __init__(self, model_name):
        self.model = AutoModelForSequenceClassification.from_pretrained(model_id)
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model.eval()

    def __call__(self, texts):
        tokenized = self.tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

        with torch.no_grad():
            outputs = self.model(**tokenized)
            logits = outputs.logits

        probabilities = softmax(logits, dim=-1)
        scores, labels = torch.max(probabilities, dim=-1)
        labels_str = [self.model.config.id2label[label_idx] for label_idx in labels.tolist()]

        return [{"label": label, "score": score.item()} for label, score in zip(labels_str, scores)]

In [186]:
custom_pipeline = CustomPipeline(model_id)
custom_pipeline(title_text)

[{'label': '경제', 'score': 0.9759466648101807},
 {'label': '사회', 'score': 0.8996158838272095},
 {'label': 'IT과학', 'score': 0.9589913487434387},
 {'label': '경제', 'score': 0.9769177436828613},
 {'label': '사회', 'score': 0.9514636993408203}]